<font size=6 bold=true>Jan 5, 2020 -- Part A
<br><br>Content Acquisition</font>

<font size=5 color='Olive'>For the first 5 lessons, DS will be combined with Python due to insufficiencies of Concepts</font>

In [4]:
import re
import spacy 
from spacy import displacy
import en_core_web_sm
from pathlib import Path
from IPython.display import display, Markdown, Latex 
import sys
import csv
import pandas as pd
import numpy as np
import os
from os import path
from datetime import datetime
from urllib.parse import urlparse, urljoin
import requests
import ntpath
from bs4 import BeautifulSoup, Comment
import time
from pprint import pprint 
import shutil
#!pip install tldextract
import tldextract

'''
    c=CollegeCrawl('University of Washington', 'https://www.washington.edu')
    c.GetallUrls()
'''
class CollegeCrawl():  
    """ Mandatory Input: 
            _collegeName: e.g., University of Toronto. 
                Targeted use case: input from batch file. 
            _rootUrl: e.g., https://www.uw.edu. 
                Targeted use case: input from batch file. 
                Both scheme and netloc has to be complete
            _mergedFile: merge all crawled file, one csv per url, into one consolidated csv file.
                Targeted use case: input from batch file.
            _existingUrlFile: a csv file that stores url list. 
                Two columns: url and status code. 
                If the file doesnot exist, will generate it.
                Targeted use case: input from batch file. 
        Optional Input:
            _gapInSecond: if second >0 then the number of seconds. if -1 then 30 seconds
            _maxPages: if -1 then 999 pages. if >0 then pages else 15 pages
            _capUrls:  if -1 then 10000 pages. if >0 then  _capUrls else 2000 pages. Not to grab too many
            _priorityKeywords=['apply', 'admission', 'application', 'deadline']
                These words will be used for priority ranking. 
            _saveToFolder=None. Where to save the crawled csv files. 
                If None, then will take current work directory.  
            _respectRobottxt=True, Not utilized for now. 
            _headers={'User-Agent':'Mozilla/5.0'}
    """        
     
    
    def __init__(self,
                 _collegeName, 
                 _rootUrl, 
                 _existingUrlFile, 
                 _mergedFile, 
                 _gapInSecond=1,
                 _maxPages=15,
                 _capUrls=2000,
                 _priorityKeywords=['apply', 'admission', 'application', 'deadline'],  
                 _saveToFolder=None, 
                 _respectrobottxt=True, 
                _headers={'User-Agent':'Mozilla/5.0'} ):
        """ __init__: 
        Mandatory Input: 
            _collegeName: e.g., University of Toronto. 
                Targeted use case: input from batch file. 
            _rootUrl: e.g., https://www.uw.edu. 
                Targeted use case: input from batch file. 
                Both scheme and netloc has to be complete
            _mergedFile: merge all crawled file, one csv per url, into one consolidated csv file.
                Targeted use case: input from batch file.
            _existingUrlFile: a csv file that stores url list. 
                Two columns: url and status code. 
                If the file doesnot exist, will generate it.
                Targeted use case: input from batch file. 
        Optional Input:
            _gapInSecond: if second >0 then the number of seconds. if -1 then 30 seconds
            _maxPages: if -1 then 999 pages. if >0 then pages else 15 pages
            _capUrls:  if -1 then 10000 pages. if >0 then  _capUrls else 2000 pages. Not to grab too many
            _priorityKeywords=['apply', 'admission', 'application', 'deadline']
                These words will be used for priority ranking. 
            _saveToFolder=None. Where to save the crawled csv files. 
                If None, then will take current work directory.  
            _respectRobottxt=True, Not utilized for now. 
            _headers={'User-Agent':'Mozilla/5.0'}
        """
            
        self.collegeName=_collegeName
         
        if urlparse(_rootUrl).scheme=="":
            print('URL needs to have scheme. Please try again.')
            raise Exception('URL needs to have a scheme')          
        
        self.rootUrl=_rootUrl 
        self.existingUrlFile=_existingUrlFile
        self.base_Domain=tldextract.extract(self.rootUrl).domain
        self.scheme=urlparse(self.rootUrl).scheme
        self.priorityKeywords=_priorityKeywords
        self.respectRobottext=_respectrobottxt
        
        if _saveToFolder==None or path.isdir(_saveToFolder)==False:
            self.saveToFolder=os.getcwd()
        else:
            self.saveToFolder=_saveToFolder
        self.mergedFile=_mergedFile     
        self.gapInSecond=_gapInSecond if _gapInSecond>0 else 30 if _gapInSecond==-1 else 0 
        self.maxPages=_maxPages if _maxPages>0 else 999 if _maxPages==-1 else 15    
        self.capUrls=_capUrls if _capUrls>0 else 10000 if _maxPages==-1 else 2000    
        self.allUrls={}
        self.headers=_headers
        self.files=[]
               
    def __str__(self):
        return '{}. Starting URL: {}'.format(self.collegeName, self.rootUrl)
    '''
        load _existingUrlFile: two columns -- Url and status_code
        minimum assumptions: first two columns are url and status_code
    '''
    
    def Load_DiscoveredUrls(self, delimiter='\t', hasHeader=False, header_names=['url', 'status_code']):   
        """Load urls list (two columns: url(str) and status_code(str)) from self.existingUrlFile. 
           If the file existingUrlFile exists, then read. if not, load dict()
           Parameters: 
               delimiter: by default is tab
               hasHeader: the csv file has header
               header_names: if any, url, status_code 
        """
        fullUrlFile=path.join(self.saveToFolder, self.existingUrlFile)
        
        if not path.exists(fullUrlFile): 
            return dict()
        else: 
                df_urls=pd.read_csv(fullUrlFile,  delimiter=delimiter, names=['url', 'status_code'])
                df_urls=df_urls[df_urls['url']!='url']  
                return dict(zip(df_urls['url'], df_urls['status_code']))   
                       
             
    def GetallUrls(self, headers=None, links_only=True): 
        """
           Get all urls from a college. 
           It carries the following steps: 
               1. Load_DiscoveredUrls
               2. SaveToCsv_FromResponse
               3. Show_Summaries(save=True) 
           visited url by priority of key words. 
           
           Parameters: 
               headers: headers from web request. 
               links_only: only handle links.            
        """ 
        urls=self.Load_DiscoveredUrls() 
        
        if len(urls)==0:
            urls={self.rootUrl:'0'}  
            unvisited=[self.rootUrl]
        else:
            unvisited=[url for url, status_code in urls.items() if str(status_code)=='0'] 
            
            if len(unvisited)==0:
                unvisited=[self.rootUrl] 
            else:
                unvisited=sorted(unvisited, key=lambda item: (sum([w in item for w in self.priorityKeywords])*20+10)/len(item))
                
        if headers==None: 
            if self.headers:
                headers=self.headers
            else: 
                 headers={'User-Agent':'Mozilla/5.0'}  
        
        if self.base_Domain==None:
            self.base_Domain=urlparse(unvisited[0]).netloc
        if self.scheme==None:
            self.scheme=urlparse(unvisited[0]).scheme
       
        pages_visited =0
        
        total0and200=len([url for url, status_code in urls.items() if str(status_code)=='0' or str(status_code)=='200'])
        print('{}\nTotal unvisited and successfully visited pages is {}. The max is approx. {}'.format(self.collegeName, total0and200, self.capUrls))
        try:  
            while len(unvisited)>0:        
                pages_visited+=1
                url=unvisited.pop()    
                url_parsed=urlparse(url)
                url_domain_path='{uri.scheme}://{uri.netloc}'.format(uri=url_parsed)
              
                response=requests.get(url, headers=headers)     
                status_code=response.status_code
                
                urls[url]=str(status_code)
                 
                if status_code==200: 
                    soup=BeautifulSoup(response.text,  'lxml') #'html.parser')  
                    
                    if total0and200<self.capUrls:                        
                        for link in soup.find_all('a'):  
                            if link.has_attr('href'):
                                link_url=link['href']    

                                if not re.match('.+@.+', link_url ):
                                    link_url=re.sub(r'[\/_+!@#$?\\\s]+$', '', link_url)

                                    parsed_uri_path=urlparse(link_url) 
                                    extract_uri_domain=tldextract.extract(link_url)                            
                                    #parts: parsed_uri.domain + '.' + parsed_uri.suffix  
                                    absolute_url=''                                 
                                    if  (extract_uri_domain.domain=='') and re.match(r'^\/.*\w$', parsed_uri_path.path) :
                                        absolute_url=urljoin(url_domain_path,parsed_uri_path.path)
                                    elif extract_uri_domain.domain==self.base_Domain: # and re.match('^http', parsed_uri.scheme):
                                        if re.match(r'^\/\/', link_url):
                                            absolute_url=self.scheme+':'+link_url
                                        else:
                                            absolute_url=link_url
                                    else:
                                        continue    

                                    if absolute_url!='' and absolute_url not in urls:   
                                         urls[absolute_url]='0' 
                    self.SaveToCsv_FromResponse(url, response)  
                
                if pages_visited>=self.maxPages:
                    break   
                
                unvisited=[name for name, code in urls.items() if str(code)=='0']    
                
                unvisited=sorted(unvisited, key=lambda item: (sum([w in item for w in self.priorityKeywords])*10+10)/len(item))
                if self.gapInSecond>0:
                    time.sleep(self.gapInSecond)  
   
        except: # Exception as e: 
            #print('url "{}" went wrong. Error {} -- {}'.format(url,e.errno, e.strerror))   
            urls[url]='999' 
            pass
        finally: 
            self.allUrls=urls  
         
    def Show_Summaries(self, show=True): 
        """
            It will take one parameter: show
            if show: 
                print out action summary
            if not show: 
                return summary dataframe. this dataframe is digested by batch file to get stats. 
        """
        if self.allUrls:
            df_urls=pd.DataFrame([[i, k] for i, k in (zip(self.allUrls.keys(), self.allUrls.values()) )], \
                                 columns=['url', 'status_code'])
            
            #df_urls=pd.DataFrame(list(c.allUrls.items()), columns=['url', 'status_code'])
            
             
            try:
                with open(self.existingUrlFile, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
                    writer.writerow(['url','status_code'])
                    for url, status_code in self.allUrls.items():  
                        writer.writerow([url, status_code])  
            except IOError as e: 
                print (("I/O Error({0}): {1} in saving summaries").format(e.errno, e.strerror))
                pass
            finally:
                df_sum=df_urls.groupby('status_code').count().reset_index()
                if show==True:
                    print('\nSummary for college ', self.collegeName)
                    print('\nUrls Identified from college {}.'.format(self.collegeName))
                    print(df_sum)
                    print('\n')
                    print('\nThe following {} file(s) are generated. '.format(len(self.files)))
                    pprint(self.files) 
                return df_sum
                  
    def Read_Oneurl(self, url):  
        """
            Get a response from request get call. 
            Followed by Get_Pagetext. 
        """
        response=requests.get(url, self.headers)  
        if response.status_code==200: 
            return self.Get_Pagetext(response)
        else: 
            return [[None, None, None, None]]
     
    def Tag_Visible(self, element):
        """
            Filter tags: 'style', 'script', 'head', 'title', 'meta', '[document]'
            This function is utilized by Get_Pagetext. 
        """
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True 
 
    def Get_Pagetext(self, response):
        """
            Get Text from the response text 
            Parameter: response. 
            Return a list of list: 
                [tagName,previousSibling, nextSibling, Text]
        """
        soup = BeautifulSoup(response.text, 'html.parser') #conent
        texts = soup.findAll(text=True) 
        visible_texts = filter(self.Tag_Visible, texts)       

        return [ [t.parent.name,   
                 t.parent.previousSibling.name if t.parent.previousSibling!=None else None, 
                 t.nextSibling.name if t.nextSibling!=None else None,
                 re.sub(r'[\s+\t]',' ',t) ]  for t in visible_texts if len(t.strip())>2]
     
    def SaveToCsv_FromUrl(self,url): 
        """
            Save text to a csv file. It calls Read_Oneurl()
            Paramter: 
                url
        """
        content=self.Read_Oneurl(url.strip()) 
        self.SaveToCsv(url, content) 
     
    def SaveToCsv_FromResponse(self, url, response): 
        """
            Save to csv file. 
            Paramters: 
                url
                response
        """
        content= self.Get_Pagetext(response)
        self.SaveToCsv(url, content) 
         
    def SaveToCsv(self, url, content):
        """
            Utility file. Save file to folder saveToFolder
            File name convention: weburl+timestr. 
            Parameters: 
                url
                content
        """
        filename=url.replace('.', '_dot_').replace('/', '_').replace(':', '_')+'_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")+'.csv'
        fullname=path.join(self.saveToFolder, filename)
        try:
            with open(fullname, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
                writer.writerow(['url', 'parent', 'ps', 'ns', 'text'])
                for lll in content: 
                    lll.insert(0, url)
                    writer.writerow(lll)  
            self.files.append(fullname) 
        except: # IOError as e:
            #print ("I/O Error({0}): {1} in saving summaries").format(e.errno, e.strerror) 
            pass
           
    def MoveFiles(self, destination_folder, source_folder=None,  pattern=None): 
        """
            Move File utility function. 
            Paramters: 
                destination_folder
                source_folder
                pattern
        """
        if source_folder==None:
            source_folder=self.saveToFolder
        if source_folder==None:
            print('Please specify folder with the files. ')
            return

        if destination_folder==None or path.isdir(destination_folder)==False:
            print('Need a valid destination folder')
            return
        if pattern==None or pattern=="":
            pattern="csv$" 
        files = [f for f in os.listdir(self.saveToFolder) if re.match(pattern, f) and path.isfile(path.join(source_folder, f))]
        for f in files:
            shutil.move(path.join(source_folder, f), destination_folder)  
    
    def Merge(self, isRemoveRawFile=False ):         
        """
            Merger files into one consolidated file. 
            drop duplicates. 
            Generate one file per college. 
            File name: supplied from constructor
            
            drop duplicates 
            use relative folder path only. ./file name
        """            
        fullFileName=path.join(self.saveToFolder,self.mergedFile) 
             
        files=[(path.join(self.saveToFolder, f), 1)  for f in os.listdir(self.saveToFolder) if re.match(r'^http.+\d+.+csv$', f)]
         
        if len(files)>0:
            if path.exists(fullFileName):
                files.append((fullFileName, 0))
            
            if len(files)==1:
                pd.read_csv(files[0][0] , sep='\t').to_csv( fullFileName, index=False, sep='\t', encoding='utf-8')
            else:            
                df_combined = pd.concat([pd.read_csv(f[0] , sep='\t') for f in files]) 

                df_combined.drop_duplicates(inplace=True)
                df_combined.to_csv( fullFileName, index=False, sep='\t', encoding='utf-8')

            if isRemoveRawFile:
                for f in files:
                    if f[1]==1:
                        os.remove(f[0])
        print('{} file(s) merged into {}\n'.format(len(files)-1, fullFileName))
 

In [5]:
display(Markdown(
"""
# About CollegeCrawl
""" ))   
print(CollegeCrawl.__doc__)


# About CollegeCrawl


 Mandatory Input: 
            _collegeName: e.g., University of Toronto. 
                Targeted use case: input from batch file. 
            _rootUrl: e.g., https://www.uw.edu. 
                Targeted use case: input from batch file. 
                Both scheme and netloc has to be complete
            _mergedFile: merge all crawled file, one csv per url, into one consolidated csv file.
                Targeted use case: input from batch file.
            _existingUrlFile: a csv file that stores url list. 
                Two columns: url and status code. 
                If the file doesnot exist, will generate it.
                Targeted use case: input from batch file. 
        Optional Input:
            _gapInSecond: if second >0 then the number of seconds. if -1 then 30 seconds
            _maxPages: if -1 then 999 pages. if >0 then pages else 15 pages
            _capUrls:  if -1 then 10000 pages. if >0 then  _capUrls else 2000 pages. Not to grab too many
            _p

In [6]:
# sample way of calling: 
if 1==11:
    c=CollegeCrawl('A.T. Still University of Health Sciences', 'http://www.atsu.edu','A_100000URL.csv', 'A_100000.csv')
    c.GetallUrls()
    summary=c.Show_Summaries(show=True)
    c.Merge(isRemoveRawFile=True)
    zero=summary[summary['status_code']=='0']['url'].sum()
    t00=summary[summary['status_code']=='200']['url'].sum()
    full=summary['url'].sum()


In [7]:
c=CollegeCrawl('Albertus Magnus College', 'http://www.albertus.edu','A_100020URL.csv', 'A_100020.csv')
     

In [12]:
k=0 
ls_sum=[]
c.GetallUrls()
summary=c.Show_Summaries(show=False)
if len(c.files)>0:
    c.Merge(isRemoveRawFile=True)
else:
    print('({}) {} : No files writen'.format(c.collegeName, c.rootUrl))
    
try:
    dataFile=path.join(c.saveToFolder, filename)
    fileSizeinMB=path.getsize(dataFile)/1024/1024
except: 
    fileSizeinMB=0.0

zero=summary[summary['status_code']=='0']['url'].sum()
t00=summary[summary['status_code']=='200']['url'].sum()
full=summary['url'].sum()
ls_sum.append([3, full,  t00, full-zero-t00, "{0:.2f}".format(fileSizeinMB)])  


Albertus Magnus College
Total unvisited and successfully visited pages is 0. The max is approx. 2000
(Albertus Magnus College) http://www.albertus.edu : No files writen
